In [32]:
import pandas as pd
import numpy as np
import re
import os
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

In [55]:
dir_name = 'DATA/1-8/1-8.xlsx'      

def rep_part(ls):
    """Удаляет .0 в номере партии и возвращает список без него"""
    for x in ls:
        x = str(x)
        x = x.replace('.0', '')
    return ls

def gen_batch_number_list(files_dict):
    batch_number_list = []
    for x in files_dict['№ партии.1']:
        batch_number_list.append(str(x).replace('.0', ''))

    batch_number_list = rep_part(batch_number_list)
    print(len(batch_number_list))
    return batch_number_list

def print_file_sizes(valid_files):
    # выводим размерности всех файлов
    print(valid_files.shape)

valid_files = pd.read_excel(dir_name)

def clean_dict(files_dict):
    # убираем заголовки
    tmp = files_dict.copy()
    tmp.columns = range(files_dict.shape[1])
    return tmp

def return_headers(df, headers_list, batch_list):
    """Возвращаем заголовки и номера партий, удаляем партии без номеров"""
    df.columns = headers_list[:len(df.columns)]
    df['№ партии'] = batch_list
    df = df[df['№ партии'] != 'nan']
    print(df.shape)
    return df

def get_df_modes(dict_modes):
    # Сохраняем столбец с номером партии в переменную batch_number_list 
    batch_number_list = gen_batch_number_list(dict_modes)
    
     # убираем последние пять столбцов из всех таблиц
    dict_modes.drop(list(dict_modes.columns[-5:-2])+[dict_modes.columns[-1]], axis=1, inplace=True)
    
    # Сохраняем заголовки
    header = dict_modes.columns
    # удаляем заголовки и лишние столбцы
    dict_modes_tmp_copy = clean_dict(dict_modes)
    df_modes = pd.DataFrame()
    # слияние всех режимов
    df_modes = pd.concat([df_modes,dict_modes_tmp_copy])
    # Сбрасываем индекс, чтобы не было повторяющихся
    df_modes.reset_index(drop=True, inplace=True)
#     print(df_modes.shape)
    # Возвращаем заголовки и номера партий, удаляем партии без номеров
    df_modes = return_headers(df_modes, header, batch_number_list)
    return df_modes


df_valid = get_df_modes(valid_files)
print(df_valid.shape, 'get_df_modes')

# исходная таблица без пакетов
df_valid = df_valid.drop(['№ пакета'], axis=1)

# Удаляем партии без номера плавки
df_valid = df_valid[df_valid['плавка'] != 'nan']
print(df_valid.shape, 'Удаляем партии без номера плавки')

# Разносим шаг на закалочной и отпускной печах в разные столбцы и очищаем их
def clean_steps(x):
    x = str(x)
    x = x.replace(u'X', '/')
    x = x.replace(u'x', '/')
    x = x.replace(u'Х', '/')
    x = x.replace(u'х', '/')
    x = x.replace(',', '.')
    x = x.replace('\\', '/')
    x = x.replace('/-', '/0')
    x = x.replace('-/', '0/')
    x = x.replace('***', '0')
    x = x.replace('*', '0')
    x = x.replace('-', '/')
    x = x.replace('nan', '')
    return x

def convert_step0(x):
    return clean_steps(x).split('/')[0]
   
def convert_step1(x):
    try:
        return clean_steps(x).split('/')[1]
    except:
        return x

# df_modes = df_modes[df_modes[u'шаг балок закалочная/отпускная печи, сек']!='***']
df_valid[u'шаг балок закалочная печь, сек'] = df_valid[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step0)
df_valid[u'шаг балок отпускная печь, сек'] = df_valid[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step1)

# Разделяем диаметр и толщину стенки и чистим их
def split_d(x):
    x = str(x)
    x = x.replace(',', '.')
    return re.split('х|Х|x|X', x)[0]

def split_s(x):
    x = str(x)
    x = x.replace(',', '.')
    try:
        return re.split('х|Х|x|X', x)[1]
    except:
        return x
    
def clean_d_s(x):
    x = str(x)
    x = x.replace('п', '')
    x = x.replace('П', '')
    x = x.replace('nan', '')
    x = x.replace(',', '.')
    return x


df_valid['диаметр'] = df_valid[u'D/S'].apply(split_d)
df_valid['толщина стенки'] = df_valid[u'D/S'].apply(split_s)
df_valid['толщина стенки'] = df_valid['толщина стенки'].apply(clean_d_s)
df_valid['толщина стенки'] = df_valid['толщина стенки'].apply(lambda x: x.replace('107.95', ''))
df_valid['диаметр'] = df_valid['диаметр'].apply(clean_d_s)

# Чистим Скорость прохождения трубы через спрейер, м/с
def clean_u_sp(x):
    x = str(x)
    x = x.replace(u'сек', '')
    x = x.replace(u'ек', '')
    x = x.replace(u'\мс', '')
    x = x.replace(u'm/c', '')
    x = x.replace(u'М/С', '')
    x = x.replace(u'М\С', '')
    x = x.replace(u'nan', '')
    x = x.replace(u'м/с', '')
    x = x.replace(u'м\с', '')
    x = x.replace(u',', u'.')
    x = x.replace(u'м\\', '')
    x = x.replace(u'м/', '')
    x = x.replace(u'*****', u'')
    x = x.replace(u'****', u'')
    x = x.replace(u'***', u'')
    x = x.replace(u'29.03.2017', u'')
    x = x.replace(',','.')
    x = x.replace('041','0.41')
    x = x.replace('НЕТ','')
    x = x.replace('нет','')  
    x = x.replace('м','')
    x = x.replace('021', '0.21')
    x = x.replace('1.8', '0.18')
    x = x.replace('2.0', '0.2')
    x = x.replace('012', '0.12')
    if x == '45':
        return '0.45'
    if x == '064':
        return '0.64'
    x = x.replace(u'0.7.', '0.7')
    try:
        y = x.split('-')
        x = (float(y[0])+float(y[1]))/2
    except:
        pass
    return x
    
col = 'Скорость прохождения трубы через спрейер, м/с' 
df_valid[col] = df_valid[col].apply(clean_u_sp)

# Чистим время выдержки в закалочной ванне, сек.
def clean_time_bath(x):
    x = str(x)
    x = x.replace(u'сек', '')
    x = x.replace(u' ', '')
    x = x.replace(u'.', '')
    x = x.replace(u'J55', '')
    return x
col = u'время выдержки в закалочной ванне, сек.'
df_valid[col] = df_valid[col].apply(clean_time_bath)

# Удаляем лишнии столбцы из датафрейма
l2del_modes = [
#     'Дата термообработки', 
    u'время начала т/о',u'D/S',u'шаг балок закалочная/отпускная печи, сек', u'плавка',
      '2 зона уставка закалка', '2 зона фактическая температура закалка', '2 зона фактическая температура отпуск', 
      '4 зона фактическая температура отпуск', 'кол-во, шт.'
]

l2del_valid =[
#     'Дата термообработки', 
    u'время начала т/о',u'D/S',u'шаг балок закалочная/отпускная печи, сек', u'плавка', 'кол-во, шт.']

def del_cols_df(df, l2del):
    df[u'№ плавки'] = df[u'плавка']
    df = df.drop(l2del, axis = 1)
    return df

df_valid = del_cols_df(df_valid, l2del_valid)

565
(559, 23)
(559, 23) get_df_modes
(559, 22) Удаляем партии без номера плавки


In [56]:
# Объединение марок стали
def rep_steel(x):
    x = str(x)
    x = x.replace('092ГС','09Г2С')
    x = x.replace('13 ХФА','13ХФА')
    x = x.replace('13хфа','13ХФА')
    x = x.replace('13ХФА ','13ХФА')
    x = x.replace('13хфа-3','13ХФА-3')
    x = x.replace('15Х13H2MБ','15Х13H2МБ')
    x = x.replace('15х5мфбч','15Х5МФБЧ')
    x = x.replace('18х3мфб','18Х3МФБ')
    x = x.replace('18Х3МФБ ','18Х3МФБ')
    x = x.replace('18х3ХМФБ','18Х3МФБ')
    x = x.replace('18хзмфб','18Х3МФБ')
    x = x.replace('18хмфб','18ХМФБ')
    x = x.replace('18ХМФБ ','18ХМФБ')
    x = x.replace('20А простая','20А-пр')
    x = x.replace('20А-ПР','20А-пр')
    x = x.replace('20 А','20А')    
    x = x.replace('20A','20А') 
    x = x.replace('20А ','20А')
    x = x.replace('20 АУ','20А-У')
    x = x.replace('20АУ','20А-У')
    x = x.replace('30Г2 ','30Г2')
    x = x.replace('30Г2.','30Г2')
    x = x.replace('30г2-2','30Г2-2')
    x = x.replace('30г2ф','30Г2Ф')
    x = x.replace('30г2','30Г2')
    x = x.replace('30хгма','30ХГМА')
    x = x.replace('30ХГМА1','30ХГМА-1')
    x = x.replace('30хма','30ХМА')
    x = x.replace('32г1','32Г1')
    x = x.replace('32г2','32Г2')
    x = x.replace('32х1мф-1','32Х1МФ-1')
    x = x.replace('32ха','32ХА')
    x = x.replace('37г2ф','37Г2Ф')
    x = x.replace('37г2Ф','37Г2Ф')
    x = x.replace('37Г2ф','37Г2Ф')
    x = x.replace('37Г2Ф ','37Г2Ф')
    x = x.replace('38Г2С ','38Г2С')
    x = x.replace('38г2с-4','38Г2С-4')
    x = x.replace('38Г2-С-9','38Г2С-9')
    x = x.replace('30Г2 ', '30Г2')
    return x

df_valid['марка стали'] = df_valid['марка стали'].apply(rep_steel)

ls_vtr = [
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    '1 зона ВТР и уставка отпуск',
    '2 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск'
]
# Очистка всех столбцов с температурами по втр 
def clean_vtr(x):
    x = str(x)
    x = x.replace('*****','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('-','')
    x = x.replace('50/*','')
    x = x.replace('нет','')
    x = x.replace('НЕТ','')
    x = x.replace('н/д','')
    return x


def clean_all(df, ls, apply_f):
    """Применяет к столбцам из списка ls датафрейма df метод apply_f"""
    for col in ls:
        df[col] = df[col].apply(apply_f)
    return df

df_valid = clean_all(df_valid, ls_vtr, clean_vtr)

ls_nd = [
    '3 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'Температура трубы на выгрузке из печи закалки',
    'Расход воды на закалочный спрейер, м3',
    't˚ C воды в закалочной ванне',
    'Температура трубы на выгрузке из печи отпуска',
    '1 зона ВТР и уставка отпуск',
    '2 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск'
]
# Чистим столбцы от н/д
def rep_nd(x):
    x = str(x)
    x = x.replace('н/д','')
    x = x.replace('.0', '')
    x = x.replace('НЕТ', '')
    x = x.replace('***', '')
#     x = x.replace('охлаждение','')
    return x

df_valid = clean_all(df_valid, ls_nd[:-11], rep_nd)

# Чистим время выдержки в закалочной ванне, сек.
def rep_time(x):
    x = str(x)
    x = x.replace('nan','')
    x = x.replace('НЕТ','')
    x = x.replace('нет','')
    x = x.replace('НОРМ','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('40-50','45')
    x = x.replace('сек','')
    x = x.replace('с','')
    x = x.replace(',','.')
    x = x.replace('11cек', '')
    x = x.replace('HET', '')
    return x


col ='время выдержки в закалочной ванне, сек.'
df_valid[col] = df_valid[col].apply(rep_time)

def rep_tempr(x):
    x = str(x)
    x = x.replace('******','')
    x = x.replace('*****','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('*','')
    x = x.replace('НОРМ-ИЯ', '')
    x = x.replace('норм-ция', '')
    x = x.replace('\\','/')
    x = x.replace('-','/')
    x = x.replace('<', '')
    x = x.replace('>', '')
    x = x.replace('Доотпуск', 'доотпуск')
    x = x.replace('до', '')
    x = x.replace('Бизнес Тренд', '')
    x = x.replace('БизнесТренд', '')
    x = x.replace('ВОЗДУХ', '')
    x = x.replace('Воздух', '')
    x = x.replace('воэдух', '')
    x = x.replace('высадка', '')
    x = x.replace('закалка', '')
    x = x.replace('НЕТ', '')
    x = x.replace('нет', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('нет', '')
    x = x.replace('НОРМ', '')
    x = x.replace('нормализ.', '')
    x = x.replace('нормализация', '')
    x = x.replace('воздух', '')
    x = x.replace('61702', '')
    x = x.replace('опытные.','')
    x = x.replace('на е','')
    x = x.replace('норм/ия','')
    x = x.replace('нормал/ция','')
    x = x.replace('АЛ/ЦИЯ','')
    x = x.replace(',','.')
    y = x.split('/')
    if len(y)==1:
        return x
    fl = True
    tmpy= []
    for i in y:
        if len(i) == 4:
            y.append(i[0:2])
            y.append(i[2:4])
            y.remove(i)
    for i in y:
        try:
            tmpy.append(float(i))
        except:
            tmpy.append(i)
            fl = False
    if(fl):
        y = tmpy   
        x = np.mean(y)
    if x=='0':
        x=''
    return x


def clean_and_tofloat(df, ls, apply_f_ls):
    
    for col, apply_f in zip(ls,apply_f_ls):
        df[col] = df[col].apply(apply_f)
    df.drop(df[df[col]=='отпуск'].index, inplace = True, axis = 0)
    df[col] = df[col].apply(
                lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float)
    return df

col = 't˚ C трубы после спреера'
df_valid = clean_and_tofloat(df_valid, [col], [rep_tempr])

def rep_mass(x):
    x = str(x)
    x = x.replace('-','')
    x = x.replace(',','.')
    x = x.replace('/','')
    x = x.replace('73.02х5.51','')
    x = x.replace('4,,640','')
    x = x.replace('3.000.','3.0')
    x = x.replace('.1.440','1.44')
    x = x.replace('01б','0.16')
    return x

col = 'масса\nтн'
# df_valid = clean_and_tofloat(df_valid, [col], [rep_mass])


def rep_tempr_bath(x):
    x = str(x)
    x = x.replace('Бизнес Тренд', '')
    x = x.replace('бизнес тренд', '')
    x = x.replace('высадка.', '')
    x = x.replace('высадка', '')
    x = x.replace('ВЫСАДКА', '')
    x = x.replace('Высадка', '')
    x = x.replace('высадка', '') 
    x = x.replace('доотпуск.', 'отпуск')
    x = x.replace('доотпуск', 'отпуск')
    x = x.replace('Доотпуск', 'отпуск')
    x = x.replace('отпуск.', 'отпуск')
    x = x.replace('на отпуск', 'отпуск')
    x = x.replace('в малахи', '')
    x = x.replace('ДООТПУСК', 'отпуск')
    x = x.replace('НЕТ', '')
    x = x.replace('нет', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('НК 8500', '')
    x = x.replace('НК 9000', '')
    x = x.replace('нормализ.', '')
    x = x.replace('нормализация', '')
    x = x.replace('нормал-ция', '')
    x = x.replace('НОРМ-ИЯ', '')
    x = x.replace('норм-ция.', '')
    x = x.replace('норм-ция', '')
    x = x.replace('нор-ция', '')
    x = x.replace('нор/ция', '')
    x = x.replace('опытные.', '')
    x = x.replace('ОПЫТНЫЕ', '')
    x = x.replace('повторная', '')
    x = x.replace('повтор', '')
    x = x.replace('ПОВТОР', '')
    x = x.replace('Самара', '')
    x = x.replace('СИТЦ', '')
    x = x.replace('Трэнд', '')
    x = x.replace('ЧПТЗ', '')
    x = x.replace('ЧТПЗ', '')
    x = x.replace('30 СЕК.', '30')
    x = x.replace('2/я зак/ка', '')
    x = x.replace('ТРЕНД', '')
    x = x.replace('ПЛ.Р. Ц42/№10','')
    x = x.replace('Челябинск','')
    x = x.replace('нормал/ция', '')
    x = x.replace('HET', '')
    x = x.replace('J55', '')
    x = x.replace('J 55', '')
    x = x.replace('/', '')
    x = x.replace('ПР 22016', '')
    x = x.replace('норм j55', '')
    if x=='0':
        x=''
    if x=='.':
        x=''
    return x

col = 't˚ C трубы после ванны'
df_valid = clean_and_tofloat(df_valid, [col, col], [rep_tempr, rep_tempr_bath])


df_valid.drop(df_valid[df_valid['шаг балок закалочная печь, сек'] == '0'].index, inplace=True, axis=0)
df_valid.drop(df_valid[df_valid['шаг балок отпускная печь, сек'] == '0'].index, inplace=True, axis=0)

ls_mean_valid = df_valid.columns[3:-3]

In [57]:
df_valid.columns

Index(['№ партии', 'НД (гост, ту)', 'марка стали', 'масса\nтн',
       '1 зона по ВТР закалка', '2 зона по ВТР закалка',
       '3 зона по ВТР закалка',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', 'время выдержки в закалочной ванне, сек.',
       't˚ C трубы после ванны', '1 зона ВТР и уставка отпуск',
       '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
       '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск',
       'масса\nтн.1', 'шаг балок закалочная печь, сек',
       'шаг балок отпускная печь, сек', 'диаметр', 'толщина стенки',
       '№ плавки'],
      dtype='object')

In [58]:
def get_mean(df, ls_mean):
#     """Усредняет по [номеру плавки, партиям] и возвращает полученный датафрейм """
#     for col in ls_mean:
#         if col in df:
#             try:
#                 df[col] = df.groupby(['№ плавки', '№ партии'])[col].transform(
#                     lambda x: x.apply(
#                     lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float).mean())
#             except:
#                 print(col)
    df = df.groupby(['№ партии','№ плавки', 'марка стали', 'диаметр', 'толщина стенки','шаг балок закалочная печь, сек','шаг балок отпускная печь, сек'])['масса\nтн.1'].sum()
    return df

df_valid['№ партии'] = df_valid['№ партии'].astype(str).str.lower()
df_valid['№ плавки'] = df_valid['№ плавки'].astype(str).str.lower()


# Усреднение по номеру плавки, партиям работает долго, ждем и пьем чаёк..

# tmp_valid = get_mean(df_valid, ls_mean_valid)

# print(tmp_valid.shape, 'get_mean')

full_ls_valid = list(df_valid.columns)
full_ls_valid = full_ls_valid[-1:]+full_ls_valid[0:-1]

tmp_valid = tmp_valid[full_ls_valid]
# tmp_valid = tmp_valid.drop_duplicates()
# print(tmp_valid.shape, 'drop_duplicates')


# def th2f(x):
#     if x=='73.02*5.51':
#         return 5.51
#     if (x==' ') | (x==''):
#         return None
#     return float(x)

# tmp_valid['толщина стенки'] = tmp_valid['толщина стенки'].apply(th2f)

# tmp_valid = tmp_valid.groupby(['№ плавки', '№ партии']).first()
# tmp_valid = tmp_valid.reset_index()

# tmp_valid.to_excel('DATA/prepared/valid_jule.xlsx')

In [64]:
df_valid = df_valid.groupby(['№ партии','№ плавки', 'марка стали', 'диаметр', 'толщина стенки','шаг балок закалочная печь, сек','шаг балок отпускная печь, сек'])['масса\nтн.1'].sum()

In [68]:
df_valid.drop_duplicates(inplace=True)
df_valid.to_excel('DATA/prepared/valid_jule.xlsx')

In [69]:
df_valid.head()

№ партии  № плавки  марка стали  диаметр  толщина стенки  шаг балок закалочная печь, сек  шаг балок отпускная печь, сек
806003    8f3806    18ХМФБ       177.8    9.20            36                              36                               107.95
806004    8f3811    18ХМФБ       177.8    9.20            36                              36                                18.51
806005    8f3802    18ХМФБ       177.8    9.20            36                              36                                84.33
806006    8f3804    18ХМФБ       177.8    9.20            36                              36                                89.35
806007    8f3805    18ХМФБ       177.8    9.20            36                              36                               118.23
Name: масса\nтн.1, dtype: float64

In [38]:
valid_files = pd.read_excel('DATA/prepared/valid_jule.xlsx')

In [39]:
valid_files.shape

(102, 21)

In [19]:
def gen_batch_number_list(files_dict):
    batch_number_list = []
    for x in files_dict['№ партии.1']:
        batch_number_list.append(str(x).replace('.0', ''))

    batch_number_list = rep_part(batch_number_list)
    print(len(batch_number_list))
    return batch_number_list
b = gen_batch_number_list(valid_files)

773


In [20]:
def get_dicts_of_mode_files(dir_name):
    valid_files = pd.read_excel(dir_name)
valid_files = get_dicts_of_mode_files('DATA/июнь 20-26/20-30.xlsx')

In [29]:
valid_files.columns, valid_files.shape

(Index(['№ пакета', '№ партии', 'плавка', 'НД (гост, ту)', 'марка стали',
        'кол-во, шт.', 'масса\nтн', 'D/S', 'время начала т/о',
        'шаг балок закалочная/отпускная печи, сек', '1 зона по ВТР закалка',
        '2 зона по ВТР закалка', '3 зона по ВТР закалка',
        'Скорость прохождения трубы через спрейер, м/с',
        't˚ C трубы после спреера', 'время выдержки в закалочной ванне, сек.',
        't˚ C трубы после ванны', '1 зона ВТР и уставка отпуск',
        '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
        '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск',
        'масса\nтн.1'],
       dtype='object'), (773, 23))